In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = pd.read_csv("rasht-2024-1-28-2025-1-28-hourly.csv")

print(data.head())

features = ['temp', 'dwpt', 'wspd']

data = data[features]

                  time  temp  dwpt   rhum  prcp  snow   wdir  wspd  wpgt  \
0  2024-01-28 00:00:00   7.8   7.8  100.0   0.3   NaN  260.0  14.4   NaN   
1  2024-01-28 01:00:00   8.0   8.0  100.0   0.2   NaN  250.0  11.2   NaN   
2  2024-01-28 02:00:00   8.0   8.0  100.0   0.9   NaN  280.0  18.4   NaN   
3  2024-01-28 03:00:00   6.8   6.8  100.0   2.4   NaN  270.0  18.0   NaN   
4  2024-01-28 04:00:00   7.0   7.0  100.0   1.3   NaN  240.0  11.2   NaN   

     pres  tsun  coco  
0  1014.5   NaN   7.0  
1  1015.0   NaN   7.0  
2  1016.0   NaN   7.0  
3  1017.1   NaN   7.0  
4  1017.0   NaN   7.0  


In [5]:
features = ['temp', 'dwpt', 'wspd']
data = data[features]


scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [13]:
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), :]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

In [7]:
time_step = 100
X, Y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0], time_step, len(features))
Y = Y.reshape(Y.shape[0], 1)

In [8]:
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
Y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)

batch_size = 64
dataset = TensorDataset(X_tensor, Y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print(f'X shape: {X_tensor.shape}, Y shape: {Y_tensor.shape}')

X shape: torch.Size([8684, 100, 3]), Y shape: torch.Size([8684, 1])


In [9]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(TransformerModel, self).__init__()
        self.model_dim = model_dim
        self.embedding = nn.Linear(input_dim, model_dim)

        # Define Encoder layers
        encoder_layers = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        self.fc_out = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        src = self.embedding(src) * np.sqrt(self.model_dim)
        src = src.permute(1, 0, 2)

        transformer_out = self.transformer_encoder(src)
        output = self.fc_out(transformer_out[-1, :, :])
        return output

In [10]:
input_dim = len(features)
model_dim = 64
num_heads = 4
num_layers = 2
output_dim = 1

model = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [11]:
num_epochs = 25
for epoch in range(num_epochs):
    for X_batch, Y_batch in dataloader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, Y_batch)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/25, Loss: 0.005473834928125143
Epoch 2/25, Loss: 0.0029310949612408876
Epoch 3/25, Loss: 0.0012125479988753796
Epoch 4/25, Loss: 0.0018321557436138391
Epoch 5/25, Loss: 0.0017497326480224729
Epoch 6/25, Loss: 0.0010897053871303797
Epoch 7/25, Loss: 0.0014201643643900752
Epoch 8/25, Loss: 0.0016880849143490195
Epoch 9/25, Loss: 0.0012819429393857718
Epoch 10/25, Loss: 0.001876005087979138
Epoch 11/25, Loss: 0.0012313956394791603
Epoch 12/25, Loss: 0.0009543628548271954
Epoch 13/25, Loss: 0.0012028645724058151
Epoch 14/25, Loss: 0.0011535659432411194
Epoch 15/25, Loss: 0.0013251652708277106
Epoch 16/25, Loss: 0.001747637870721519
Epoch 17/25, Loss: 0.002167580183595419
Epoch 18/25, Loss: 0.0012257816269993782
Epoch 19/25, Loss: 0.0018977030413225293
Epoch 20/25, Loss: 0.0009659833158366382
Epoch 21/25, Loss: 0.0019104640232399106
Epoch 22/25, Loss: 0.001225130632519722
Epoch 23/25, Loss: 0.0014630878577008843
Epoch 24/25, Loss: 0.0012033688835799694
Epoch 25/25, Loss: 0.001192102

In [14]:
model.eval()
with torch.no_grad():
    predictions = model(X_tensor).cpu().numpy()


predictions_full = np.zeros((predictions.shape[0], len(features)))
predictions_full[:, 0] = predictions[:, 0]


predictions = scaler.inverse_transform(predictions_full)[:, 0]


print(predictions)

[5.68182422 6.46407229 6.37870523 ... 7.91018455 7.13411853 7.3375609 ]


In [15]:
y_true = Y_tensor.cpu().numpy()
with torch.no_grad():
    y_pred = model(X_tensor).cpu().numpy()


mse_loss = np.mean((y_true - y_pred) ** 2)
print(f'MSE Loss: {mse_loss}')

y_true = Y_tensor.cpu().numpy()
with torch.no_grad():
    y_pred = model(X_tensor).cpu().numpy()


rmse_loss = np.sqrt(np.mean((y_true - y_pred) ** 2))
print(f'RMSE Loss: {rmse_loss}')


MSE Loss: 0.0020029402803629637
RMSE Loss: 0.04475422203540802
